In [1]:
import pandas as pd
import numpy as np

In [2]:
g_sequences = {}
g_chunks_sizes = {}
digit_change = {}
g_sequences[0] = ['13524232514', '51423252413', '35421252143', '14325242135'] #Group 1 sequences
g_sequences[1] = ['13524232514', '51423252413', '41325242351', '14325242135'] #Group 2 sequences

g_chunks_sizes[0] = ['2333', '3332', '2333', '3332']  #Group1 chunking structures
g_chunks_sizes[1] = ['3332', '2333', '3332', '2333']  # Group2 chunking structures

digit_change = [5, 7, 9]

fingers = ['1', '2', '3', '4', '5']

iti = 3000 
execTime = 10000 # msecs for each trial maximum
planTime = 0 # msecs for planning before movement 
hand = 2 #left or right hand

total_sub_num = 10
# total_num_blocks = 8
num_train_blocks = 5
num_test_blocks = 5
total_num_blocks = num_train_blocks + num_test_blocks
num_trials_per_train_block = 20
num_seq_changed_trials_per_test_block = 8
num_seq_unchanged_trials_per_test_block = 8
num_random_trials_per_test_block = 8
num_trials_per_test_block = num_seq_changed_trials_per_test_block + num_seq_unchanged_trials_per_test_block + num_random_trials_per_test_block


In [3]:
def check_run(seq):
    #check runs of 3
    # for pos in range(len(seq)- 2):
    #     if (int(seq[pos]) == int(seq[pos + 1]) + 1 and int(seq[pos+1]) == int(seq[pos+2]) + 1):
    #         return True
    #     if (int(seq[pos]) == int(seq[pos + 1]) - 1 and int(seq[pos+1]) == int(seq[pos+2]) - 1):
    #         return True
    
    #check double press
    
    for pos in range(len(seq) - 1):
        if(int(seq[pos]) == int(seq[pos+1])):
            return True
        
    return False


In [4]:
check_run("153235")

False

In [5]:
## target file headers: [subNum, hand, isTrain, seq, planTime, execTime, iti, chunkSize, digitChangePos, digitChangeValue]
for sub in range(6, total_sub_num):
    # group  = int(np.random.rand() * 2)
    group = sub % 2
    subNum = sub+1
    hand = 2
    for block_num in range(1, total_num_blocks+1):
        block = pd.DataFrame(columns=["subNum", "group", "hand", "isTrain", "seq", "planTime", "execTime", "iti", "chunkSize", "digitChangePos", "digitChangeValue"])
        # if (block_num%4 == 0):
        if (block_num > num_train_blocks):
            isTrain = 0
            for trial_num in range(num_trials_per_test_block):
                chunkSize = g_chunks_sizes[group][trial_num % 4]
                digit_change_pos = np.random.choice(digit_change)
                if (int (trial_num / num_seq_changed_trials_per_test_block) == 0): ## treatment sequence change digit todo: check whether to exclude original digit from new digit
                    seq = g_sequences[group][trial_num % 4]
                    while (True):
                        digit_change_value = str(int(np.random.rand() * 5 + 1))
                        new_seq = seq[:digit_change_pos] + digit_change_value + seq[digit_change_pos + 1 :]
                        if(not check_run(new_seq)):
                            break
                if (int (trial_num / num_seq_changed_trials_per_test_block) == 1): ## treatment sequence unchanged
                    seq = g_sequences[group][trial_num % 4] 
                    while (True):
                        digit_change_value = seq[digit_change_pos]
                        new_seq = seq[:digit_change_pos] + digit_change_value + seq[digit_change_pos + 1 :]
                        if(not check_run(new_seq)):
                            break
                
                if (int (trial_num / num_seq_changed_trials_per_test_block) == 2): ## random sequence ##todo: check whether need to exclude treatment seq from random sequence
                    while (True):
                        seq = ''.join(np.random.choice(fingers, 11))
                        digit_change_value = str(int(np.random.rand() * 5 + 1))
                        new_seq = seq[:digit_change_pos] + digit_change_value + seq[digit_change_pos + 1 :]
                        if(not check_run(new_seq)):
                            break
                    
                trial  = pd.DataFrame([[subNum, group, hand, isTrain, seq, planTime, execTime, iti, chunkSize, digit_change_pos, digit_change_value]], columns = block.columns)
                block = pd.concat([block, trial], ignore_index=True)

        
        else:
            isTrain = 1
            for trial_num in range(num_trials_per_train_block):
                seq = g_sequences[group][trial_num % 4]
                chunkSize = g_chunks_sizes[group][trial_num % 4]
                digit_change_pos = np.random.choice(digit_change)
                while (True):
                    digit_change_value = str(int(np.random.rand() * 5 + 1))
                    new_seq = seq[:digit_change_pos] + digit_change_value + seq[digit_change_pos + 1 :]                    
                    if(not check_run(new_seq)):
                        break
                trial  = pd.DataFrame([[subNum, group, hand, isTrain, seq, planTime, execTime, iti, chunkSize, digit_change_pos, digit_change_value]], columns = block.columns)
                block = pd.concat([block, trial], ignore_index=True)
                

        block = block.sample(frac=1).reset_index(drop = True)
        block.to_csv(f'CI1_subj{subNum}_group{group+1}_block{block_num}.tgt', index=False, sep='\t')

                

    